# Bestimmung der Transformationsmatrizen


1. Importieren der benötigten Bibliotheken
2. Definition der elementaren Transformationen
3. Dateipfade einlesen (Abgabeverzeichnis 1_Daten/2_processed)
4. Transformationsbestimmungsfunktionen definieren
5. Werte für alle Dateien bestimmen und abspeichern

In [1]:
# Schritt 1: Importieren der benötigten Bibliotheken

import sys
import os
import pandas as pd
import numpy as np
import math
from ipywidgets import Button, interactive, IntSlider, FloatSlider
from IPython.display import display, clear_output
from ipywidgets import interactive, IntSlider, FloatSlider
sys.path.append('../3_Lib/py')
from helper import plot, plot_with_image, plot_vectors, plot_vectors_with_image, plot_vectors_with_image_and_aois, plot_heatmap


In [2]:
# Schritt 2: Definition der elementaren Transformationen

# siehe https://de.wikipedia.org/wiki/Abbildungsmatrix
#       https://de.wikipedia.org/wiki/Koordinatentransformation
#       https://de.wikipedia.org/wiki/Homogene_Koordinaten
#       https://de.wikipedia.org/wiki/Drehmatrix

def translate(x, y, z):
    return np.array([
        [1, 0, 0, x],
        [0, 1, 0, y],
        [0, 0, 1, z],
        [0, 0, 0, 1]
    ])

def rotate_x(phi):
    return np.array([
        [1, 0,        0,         0],
        [0, math.cos(phi), -math.sin(phi), 0],
        [0, math.sin(phi), math.cos(phi),  0],
        [0, 0,        0,         1]
    ])

def rotate_y(phi):
    return np.array([
        [math.cos(phi),  0, math.sin(phi), 0],
        [0,         1, 0,        0],
        [-math.sin(phi), 0, math.cos(phi), 0],
        [0,         0, 0,        1]
    ])

def rotate_z(phi):
    return np.array([
        [math.cos(phi), -math.sin(phi), 0, 0],
        [math.sin(phi), math.cos(phi),  0, 0],
        [0,        0,         1, 0],
        [0,        0,         0, 1]
    ])

def scale_x(factor):
    return np.array([
        [factor, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ])

def scale_y(factor):
    return np.array([
        [1, 0, 0, 0],
        [0, factor, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ])

def scale_z(factor):
    return np.array([
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, factor, 0],
        [0, 0, 0, 1]
    ])

In [31]:
# Schritt 3: Dateipfade einlesen (Abgabeverzeichnis 1_Daten/2_processed)

# siehe https://stackoverflow.com/questions/19587118/iterating-through-directories-with-python

root = '../1_Daten/2_processed'
FILE_NAMES = []

for subdir, dirs, files in os.walk(root):
    for file in files:
        if 'annotation' not in file or '.csv' not in file:
            continue
        FILE_NAMES.append(os.path.join(subdir, file).replace('\\', '/'))        

In [187]:
# Schritt 4: Transformationsbestimmungsfunktionen definieren

class CameraValues(object):
    def __init__(self):
        self.RX = 0
        self.RY = 90
        self.RZ = 30
        self.SX = 0.4
        self.SY = 0.4
        self.SZ = 0.4
        self.DX = 5
        self.DY = 215
        self.DZ = 680
        
def test_and_save_camera_transformation(coordinates, client_id):
    ORIGIN = np.array([[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1]], dtype=float)
    AXES = np.array([[1000, 0, 0, 1], [0, 1000, 0, 1], [0, 0, 1000, 1]], dtype=float)
    AXES_COLORS = [['r', 'k'], ['g', 'k'], ['b', 'k']]
    
    export_button = Button(description="Store to config.json")
    
    cv = CameraValues()

    def draw(cv, rx, ry, rz, sx, sy, sz, dx, dy, dz):
        cv.RX, cv.RY, cv.RZ, cv.SX, cv.SY, cv.SZ, cv.DX, cv.DY, cv.DZ = rx, ry, rz, sx, sy, sz, dx, dy, dz
        rot_x = rotate_x(math.radians(rx))
        rot_y = rotate_y(math.radians(ry))
        rot_z = rotate_z(math.radians(rz))

        s_x = scale_x(sx)
        s_y = scale_y(sy)
        s_z = scale_z(sz)

        d = translate(dx, dy, dz)

        tr = scale_x(-1) @ rotate_y(math.radians(-90)) @ d @ s_z @ s_y @ s_x @ rot_z @ rot_y @ rot_x @ scale_y(-1)

        plot_vectors_with_image(np.swapaxes(ORIGIN @ tr.T, 0, 1), np.swapaxes(AXES @ tr.T, 0, 1), 'Kameratest', colors=AXES_COLORS, line_width=3)
        plot_with_image(np.swapaxes(coordinates @ tr.T, 0, 1), 'Transformierte Koordinaten')

    interactive_plot_cam_1 = interactive(
        lambda rx, ry, rz, sx, sy, sz, dx, dy, dz: draw(cv, rx, ry, rz, sx, sy, sz, dx, dy, dz),
        rx=IntSlider(min=-180, max=180, step=1, value=0),
        ry=IntSlider(min=-180, max=180, step=1, value=90),
        rz=IntSlider(min=-180, max=180, step=1, value=40),
        sx=FloatSlider(min=-0.01, max=1, step=0.01, value=0.4),
        sy=FloatSlider(min=-0.01, max=1, step=0.01, value=0.4),
        sz=FloatSlider(min=-0.01, max=1, step=0.01, value=0.4),
        dx=IntSlider(min=-1000, max=1000, step=1, value=5),
        dy=IntSlider(min=-1000, max=1000, step=1, value=200),
        dz=IntSlider(min=-1000, max=1000, step=1, value=720)
    )
    
    interactive_plot_cam_2 = interactive(
        lambda rx, ry, rz, sx, sy, sz, dx, dy, dz: draw(cv, rx, ry, rz, sx, sy, sz, dx, dy, dz),
        rx=IntSlider(min=-180, max=180, step=1, value=0),
        ry=IntSlider(min=-180, max=180, step=1, value=90),
        rz=IntSlider(min=-180, max=180, step=1, value=40),
        sx=FloatSlider(min=-0.01, max=1, step=0.01, value=0.4),
        sy=FloatSlider(min=-0.01, max=1, step=0.01, value=0.4),
        sz=FloatSlider(min=-0.01, max=1, step=0.01, value=0.4),
        dx=IntSlider(min=-1000, max=1000, step=1, value=5),
        dy=IntSlider(min=-1000, max=1000, step=1, value=200),
        dz=IntSlider(min=-1000, max=1000, step=1, value=535)
    )
    
    interactive_plot_cam_3 = interactive(
        lambda rx, ry, rz, sx, sy, sz, dx, dy, dz: draw(cv, rx, ry, rz, sx, sy, sz, dx, dy, dz),
        rx=IntSlider(min=-180, max=180, step=1, value=0),
        ry=IntSlider(min=-180, max=180, step=1, value=90),
        rz=IntSlider(min=-180, max=180, step=1, value=40),
        sx=FloatSlider(min=-0.01, max=1, step=0.01, value=0.4),
        sy=FloatSlider(min=-0.01, max=1, step=0.01, value=0.4),
        sz=FloatSlider(min=-0.01, max=1, step=0.01, value=0.4),
        dx=IntSlider(min=-1000, max=1000, step=1, value=5),
        dy=IntSlider(min=-1000, max=1000, step=1, value=200),
        dz=IntSlider(min=-1000, max=1000, step=1, value=350)
    )
    
    interactive_plot_cam_4 = interactive(
        lambda rx, ry, rz, sx, sy, sz, dx, dy, dz: draw(cv, rx, ry, rz, sx, sy, sz, dx, dy, dz),
        rx=IntSlider(min=-180, max=180, step=1, value=0),
        ry=IntSlider(min=-180, max=180, step=1, value=90),
        rz=IntSlider(min=-180, max=180, step=1, value=40),
        sx=FloatSlider(min=-0.01, max=1, step=0.01, value=0.4),
        sy=FloatSlider(min=-0.01, max=1, step=0.01, value=0.4),
        sz=FloatSlider(min=-0.01, max=1, step=0.01, value=0.4),
        dx=IntSlider(min=-1000, max=1000, step=1, value=5),
        dy=IntSlider(min=-1000, max=1000, step=1, value=200),
        dz=IntSlider(min=-1000, max=1000, step=1, value=165)
    )
    
    if(client_id == "cam_1"):
        return export_button, interactive_plot_cam_1
    if(client_id == "cam_2"):
        return export_button, interactive_plot_cam_2
    if(client_id == "cam_3"):
        return export_button, interactive_plot_cam_3
    if(client_id == "cam_4"):
        return export_button, interactive_plot_cam_4    
    
    return export_button, interactive_plot_cam_1

In [188]:
# Schritt 5: Werte für alle Dateien bestimmen und abspeichern

def get_next_file(client_id):
    main_frame = pd.DataFrame()

    for f in FILE_NAMES:
        df = pd.read_csv(f, sep=',', engine='python')
        main_frame = main_frame.append(df[(df['client_id'] == client_id)])
        
    coords = np.array([main_frame['eye_l_X'], main_frame['eye_l_Y'], main_frame['eye_l_Z']])
    gazes = np.array([main_frame['gaze_l_X'], main_frame['gaze_l_Z'], main_frame['gaze_l_Y']])

    coords = np.insert(np.swapaxes(coords, 0, 1), 3, 1, axis=1)
    gazes = np.insert(np.swapaxes(coords, 0, 1), 3, 1, axis=1)
    
    print("Erkannte Blicke: " + str(len(main_frame)))
                    
    return True, coords, gazes
    
    
def handle_exported(client_id):
    clear_output()
    success, coordinates, gazes = get_next_file(client_id)
    if success == True:
        b1, interactive = test_and_save_camera_transformation(coordinates, client_id)
        display(b1, interactive)
        


## Kamera 1 - id: client_1

In [189]:
handle_exported("cam_1")

Erkannte Blicke: 2158


Button(description='Store to config.json', style=ButtonStyle())

interactive(children=(IntSlider(value=0, description='rx', max=180, min=-180), IntSlider(value=90, description…

## Kamera 2 - id: client_2

In [190]:
handle_exported("cam_2")

Erkannte Blicke: 2580


Button(description='Store to config.json', style=ButtonStyle())

interactive(children=(IntSlider(value=0, description='rx', max=180, min=-180), IntSlider(value=90, description…

## Kamera 3 - id: client_3

In [191]:
handle_exported("cam_3")

Erkannte Blicke: 2164


Button(description='Store to config.json', style=ButtonStyle())

interactive(children=(IntSlider(value=0, description='rx', max=180, min=-180), IntSlider(value=90, description…

## Kamera 4 - id: client_4

In [192]:
handle_exported("cam_4")

Erkannte Blicke: 4218


Button(description='Store to config.json', style=ButtonStyle())

interactive(children=(IntSlider(value=0, description='rx', max=180, min=-180), IntSlider(value=90, description…